# Some manipulations on (Kahraman, 1994)

[1] A. Kahraman, "Natural Modes of Planetary Gear Trains", Journal of Sound and Vibration, vol. 173, no. 1, pp. 125-130, 1994. https://doi.org/10.1006/jsvi.1994.1222.

In [19]:
from sympy import *
init_printing()

def symb(x,y):
    return symbols('{0}_{1}'.format(x,y), type = float)

# Displacement vector:

In [20]:
n = 3 # number of planets
N = n + 3 # number of degrees of freedom
crs = ['c', 'r', 's'] # carrier, ring, sun
pla = ['p{}'.format(idx + 1) for idx in range(n)] # planet
crs = crs + pla # put them together

coeff_list = symbols(crs)
c = coeff_list[0]
r = coeff_list[1]
s = coeff_list[2]

X = Matrix([symb('u', v) for v in coeff_list])
coeff_list[3:] = symbols(['p']*n)
p = coeff_list[3]
X.transpose() # Eq. (1a)

[u_c  uᵣ  uₛ  uₚ₁  uₚ₂  uₚ₃]

## Stiffness matrix:
![alt text](stiffness_matrix.png)

where:
* $k_1$: mesh stiffness for the ring-planet gear pair
* $k_2$: mesh stiffness for the sun-planet gear pair
* $k_c$: carrier housing stiffness
* $k_r$: ring housing stiffness
* $k_s$: sun housing stiffness
* Diagonal 1, in red
* Diagonal 2, in grey
* Off-diagonal, in blue

In [21]:
k_1, k_2, k_c, k_r, k_s = symbols('k_1 k_2 k_c k_r k_s', type = float)

# Diagonal 1:
K_d1 = zeros(3, 3)

K_d1[0, 0] = n*(k_1 + k_2) + k_c
K_d1[1, 1] = n* k_1 + k_r
K_d1[2, 2] = n* k_2 + k_s
K_d1[0, 1] = K_d1[1, 0] = -n*k_1
K_d1[0, 2] = K_d1[2, 0] = -n*k_2

# Diagonal 2:
K_d2 = eye(n)*(k_1 + k_2)

# Off diagonal:
K_od = zeros(n, n)
K_od[:, 0] = (k_1 - k_2)*ones(n, 1)
K_od[:, 1] = -k_1       *ones(n, 1)
K_od[:, 2] =        k_2 *ones(n, 1)

K = BlockMatrix([[K_d1, K_od.transpose()],
                 [K_od, K_d2]])
K = Matrix(K)

if(not K.is_symmetric()):
    print('error.')
K

⎡3⋅k₁ + 3⋅k₂ + k_c    -3⋅k₁      -3⋅k₂    k₁ - k₂  k₁ - k₂  k₁ - k₂⎤
⎢                                                                  ⎥
⎢      -3⋅k₁        3⋅k₁ + kᵣ      0        -k₁      -k₁      -k₁  ⎥
⎢                                                                  ⎥
⎢      -3⋅k₂            0      3⋅k₂ + kₛ    k₂       k₂       k₂   ⎥
⎢                                                                  ⎥
⎢     k₁ - k₂          -k₁        k₂      k₁ + k₂     0        0   ⎥
⎢                                                                  ⎥
⎢     k₁ - k₂          -k₁        k₂         0     k₁ + k₂     0   ⎥
⎢                                                                  ⎥
⎣     k₁ - k₂          -k₁        k₂         0        0     k₁ + k₂⎦

## Inertia matrix:

In [22]:
M = diag(*[symb('m', v) for v in coeff_list])
M

⎡m_c  0   0   0   0   0 ⎤
⎢                       ⎥
⎢ 0   mᵣ  0   0   0   0 ⎥
⎢                       ⎥
⎢ 0   0   mₛ  0   0   0 ⎥
⎢                       ⎥
⎢ 0   0   0   mₚ  0   0 ⎥
⎢                       ⎥
⎢ 0   0   0   0   mₚ  0 ⎥
⎢                       ⎥
⎣ 0   0   0   0   0   mₚ⎦

## Remove ring degree of freedom

In [23]:
X.row_del(1)
K.row_del(1)
K.col_del(1)
M.row_del(1)
M.col_del(1)
coeff_list.remove(r)
N = N - 1

## Coordinate transformation:

First from translational to torsional coordinates, them making the sun DOF to be the last one, making it easier to assemble a multi-stage gearbox.

In [24]:
R_1 = diag(*[symb('r', v) for v in coeff_list])
R_1

⎡r_c  0   0   0   0 ⎤
⎢                   ⎥
⎢ 0   rₛ  0   0   0 ⎥
⎢                   ⎥
⎢ 0   0   rₚ  0   0 ⎥
⎢                   ⎥
⎢ 0   0   0   rₚ  0 ⎥
⎢                   ⎥
⎣ 0   0   0   0   rₚ⎦

making the sun DOF to be the last one:

In [25]:
N1 = N - 1

R_2 = zeros(N, N)
R_2[0, 0] = 1
R_2[1, N1] = 1
R_2[2:N, 1:N1] = eye(n)
R_2

⎡1  0  0  0  0⎤
⎢             ⎥
⎢0  0  0  0  1⎥
⎢             ⎥
⎢0  1  0  0  0⎥
⎢             ⎥
⎢0  0  1  0  0⎥
⎢             ⎥
⎣0  0  0  1  0⎦

In [26]:
R = R_1*R_2
RMR = lambda m: transpose(R)*m*R

### Inertia matrix

In [27]:
M = RMR(M)

if(not M.is_symmetric()):
    print('error in M matrix')
M

⎡       2                                ⎤
⎢m_c⋅r_c     0       0       0       0   ⎥
⎢                                        ⎥
⎢               2                        ⎥
⎢   0      mₚ⋅rₚ     0       0       0   ⎥
⎢                                        ⎥
⎢                       2                ⎥
⎢   0        0     mₚ⋅rₚ     0       0   ⎥
⎢                                        ⎥
⎢                               2        ⎥
⎢   0        0       0     mₚ⋅rₚ     0   ⎥
⎢                                        ⎥
⎢                                       2⎥
⎣   0        0       0       0     mₛ⋅rₛ ⎦

### Stiffness matrix

In [28]:
K = RMR(K)

if(not K.is_symmetric()):
    print('error in K matrix')

The housing stiffness for both carrier and sunare null:

In [29]:
K = K.subs([(k_c, 0), (k_s, 0)])
K

⎡   2                                                                         
⎢r_c ⋅(3⋅k₁ + 3⋅k₂)  r_c⋅rₚ⋅(k₁ - k₂)  r_c⋅rₚ⋅(k₁ - k₂)  r_c⋅rₚ⋅(k₁ - k₂)  -3⋅
⎢                                                                             
⎢                       2                                                     
⎢ r_c⋅rₚ⋅(k₁ - k₂)    rₚ ⋅(k₁ + k₂)           0                 0            k
⎢                                                                             
⎢                                         2                                   
⎢ r_c⋅rₚ⋅(k₁ - k₂)          0           rₚ ⋅(k₁ + k₂)           0            k
⎢                                                                             
⎢                                                           2                 
⎢ r_c⋅rₚ⋅(k₁ - k₂)          0                 0           rₚ ⋅(k₁ + k₂)      k
⎢                                                                             
⎢                                                   

From that, one can write the matrices for a planetary system with $n$-planets using the following code:

In [30]:
m_c, m_s, m_p, r_c, r_s, r_p = symbols('m_c m_s m_p r_c r_s r_p', type = float)

M_p = zeros(N, N)
M_p[0, 0] = m_c*r_c**2
M_p[N1, N1] = m_s*r_s**2
M_p[1:N1, 1:N1] = m_p*r_p**2 * eye(n)

K_p = zeros(N, N)
K_p[0, 0] = n*(k_1 + k_2)*r_c**2
K_p[N1, 0] = -n*k_2*r_s*r_c
K_p[0, N1] = -n*k_2*r_s*r_c
K_p[N1, N1] = n*k_2*r_s**2
K_p[0, 1:N1] = (k_1 - k_2)*r_c*r_p*ones(1, n)
K_p[1:N1, 0] = (k_1 - k_2)*r_c*r_p*ones(n, 1)
K_p[N1, 1:N1] = k_2*r_p*r_s*ones(1, n)
K_p[1:N1, N1] = k_2*r_p*r_s*ones(n, 1)
K_p[1:N1, 1:N1] = (k_1 + k_2)*r_p**2 * eye(n)

m_diff = abs(matrix2numpy(simplify(M_p - M))).sum()
k_diff = abs(matrix2numpy(simplify(K_p - K))).sum()

if(m_diff != 0.0):
    print('Error in M matrix.')

if(k_diff != 0.0):
    print('Error in K matrix.')

## Combining planet DOFs:

In [31]:
C = zeros(N, 3)
C[   0, 0] = 1
C[  N1, 2] = 1
C[1:N1, 1] = ones(n, 1)

CMC = lambda m: transpose(C)*m*C

### Inertia matrix

In [32]:
M_C = CMC(M)

if(not M_C.is_symmetric()):
    print('error in M_C matrix')
M_C

⎡       2                  ⎤
⎢m_c⋅r_c      0        0   ⎥
⎢                          ⎥
⎢                 2        ⎥
⎢   0      3⋅mₚ⋅rₚ     0   ⎥
⎢                          ⎥
⎢                         2⎥
⎣   0         0      mₛ⋅rₛ ⎦

### Stiffness matrix

In [33]:
K_C = CMC(K)

if(not K_C.is_symmetric()):
    print('error in M_C matrix')
K_C

⎡   2                                                ⎤
⎢r_c ⋅(3⋅k₁ + 3⋅k₂)  3⋅r_c⋅rₚ⋅(k₁ - k₂)  -3⋅k₂⋅r_c⋅rₛ⎥
⎢                                                    ⎥
⎢                         2                          ⎥
⎢3⋅r_c⋅rₚ⋅(k₁ - k₂)   3⋅rₚ ⋅(k₁ + k₂)     3⋅k₂⋅rₚ⋅rₛ ⎥
⎢                                                    ⎥
⎢                                                 2  ⎥
⎣   -3⋅k₂⋅r_c⋅rₛ         3⋅k₂⋅rₚ⋅rₛ        3⋅k₂⋅rₛ   ⎦

## Adapting it to a parallel gear set

Considering only one of the sun-planets pairs, one should change the sub-indexes in the following way:
* [p]lanet => [w]heel
* [s]un    => [p]inion;
It also necessary to remove the mesh stiffness of the ring-planet pair
### Inertia matrix

In [34]:
k, w, p = symbols('k w p', type = float)
m_w, m_p, r_w, r_p = symbols('m_w m_p r_w r_p', type = float)

N2 = N - 2
M_par = M[N2:, N2:]
M_par = M_par.subs([(m_p, m_w), (m_s, m_p), (r_p, r_w), (r_s, r_p)]) # 
M_par

⎡       2        ⎤
⎢m_w⋅r_w     0   ⎥
⎢                ⎥
⎢               2⎥
⎣   0      mₚ⋅rₚ ⎦

### Stiffness matrix

In [35]:
K_par = K[N2:, N2:]

K_par = K_par.subs(k_1, 0) # ring-planet mesh stiffness
K_par = K_par.subs(k_s, 0) # sun's bearing stiffness
K_par = K_par.subs(n*k_2, k_2) # only one pair, not n
K_par = K_par.subs(k_2, k) # mesh-stiffness of the pair
K_par = K_par.subs([(r_p, r_w), (r_s, r_p)])
K_par

⎡      2           ⎤
⎢ k⋅r_w    k⋅rₚ⋅r_w⎥
⎢                  ⎥
⎢               2  ⎥
⎣k⋅rₚ⋅r_w   k⋅rₚ   ⎦

From that, one can write the matrices for a parallel system using the following code:

In [36]:
M_p = diag(m_w*r_w**2, m_p*r_p**2)

mat_diff = abs(matrix2numpy(simplify(M_p - M_par))).sum()
if(mat_diff != 0.0):
    print('Error in M_p matrix.')

K_p       = diag(r_w**2, r_p**2)
K_p[0, 1] = r_p*r_w
K_p[1, 0] = r_p*r_w
K_p       = k*K_p

mat_diff = abs(matrix2numpy(simplify(K_p - K_par))).sum()
if(mat_diff != 0.0):
    print('Error in K_p matrix.')